# Tarea 1 Grupo 24

###### Francisco Paez
###### Catalina Blanco
###### Diego Di Battista



En el siguiente informe analizaremos dos problemas relacionados con la pandemia de COVID-19 que, en la actualidad, azota a todo el mundo, enfocándonos en nuestros algoritmos para obtener las soluciones y en el por qué usamos cada parte del algoritmo.
El primer problema trata del modelo de contagio del virus, en el cual usamos bases de datos de distintos tamaños con instancias de individuos y las reuniones interpersonales en las que interactuaban cada día, y para las probabilidades de contagio de cada persona en ellas. En el problema simulamos la escalabilidad de los contagios, dados los parámetros de probabilidad de contagio y las instancias dadas de reuniones entre personas. También ilustramos graficamente curvas cumulativas tanto de contagios, recuperados y enfermos para cada día de la simulación. Seguido de eso, analizamos la eficiencia de nuestro algoritmo y lo pusimos a prueba haciendo un número de simulaciones para así obtener probabilidades frecuentistas de contagio, usando distintos datos en cada una (cambiando la semilla de simulación).

El segundo problema habla del abastecimiento y distribución provocado por las restricciones interpersonales que se han impuesto para poder frenar este virus. **(ACÁ FALTA DESCRIBIR EL SEGUNDO PROBLEMA, LO HAGO DESPUÉS)**

En la primera parte del informe, hablaremos del primer problema, del contagio, donde explicaremos los archivos secundarios del código, con sus funciones y estructuras de datos, y después pasaremos al archivo principal, en el que mostrareros mostraremos las librerías utilizadas, analizaremos la complejidad del algoritmo y resultados obtenidos a partir de lo requerido en el enunciado.


## Archivos secundarios


En primer lugar se nos fue entregada la funcion **simulation_parameters()** que nos entregará la cantidad de días que tienen que pasar para que una persona que se enfermó con el virus, vuelva a estar completamente sana. También nos dice los días que demora en darse cuenta que está infectada, es decir, la cantidad de días que es vector de contagio y no lo sabe, por lo que sigue teniendo reuniones con otras personas, pudiendo infectarlas. Por último, nos entrega **contagios** que es una especie de dado para poder trabajar como variable aleatoria la probabilidad de contagio de cada persona en cada día.
Nosotros como grupo, creemos que esta función es un tanto ineficiente, ya que nos da un nuevo número de probabilidad de contagio para cada persona cada día, aun cuando estas personas podrían no tener una reunión aquella jornada, por lo que nos da datos inútiles. Sin embargo, no es un problema para el código dado que lo tuvimos en cuenta en el orden y tampoco es un problema en términos de memoria por el tamaño de las instancias.

In [ ]:
from random import seed, random
from numpy.random import poisson, negative_binomial
from numpy.random import seed as npseed



def simulation_parameters(total, delta_t,s):
    seed(s)
    npseed(s)
    r_p = {i: poisson(15) for i in range(total)} # El tiempo de recuperacion
    c_p = {i: min(r_p[i], negative_binomial(6,0.5)) for i in range(total)} # Tiempo en que se da cuenta
    contagios = {t: {i: random() for i in range(total)} for t in range(delta_t + 1)}

    return r_p, c_p, contagios
### es ineficiente, porque nos da el numero de probabilidad de que se contagie en cada día aunque no se reunan (son numeros inuyiles)
## que no es tan importante, pero lo tuvimos en cuenta para el orden

Creamos la clase ```Persona```, que toma como parámetros el *id* de la persona, es decir, el número para identificarla, y su probabilidad de contagio dada en la base de datos de las personas. Como atributos de esta clase hicimos el atributo *id*, que está dado por su parámetro homónimo, también hicimos el atributo **.\_probabilidad_contagio** que al igual que *id* solamente viene dado por el parámetro. Creamos los atributos estado, días de recuperacón y días de síntoma para ```Persona```.
* **Estado:** describe el estado de salud de cada persona. Inicia siempre con *S* que significa que todos comienzan sanos, pero pueden haber cambios de estado debido a los contagios y avances del virus en cada persona, tema que se explicará más adelante. 
* **Días de recuperación (.\_dias_recuperacion):** es una cuenta regresiva que representa cuántos días quedan para que el infectado vuelva a estar sano. Inicia siempre en 0, debido a que, como dijimos antes, todos comienzan sanos. Vale decir que este atributo cambia de valor cuando se contagia. 
* **Días de síntomas (.\_dias_sintomas):** también es una cuenta regresiva que al igual que la anterior se inicializa en 0 por la misma razón y solo se cambia el valor cuando la persona se infecta. Sin embargo, esta representa los días restantes para que la persona comience a presentar síntomas, es decir, que no solamente sea un vector de contagio, sino que pase a ser debidamente un infectado (y en la situación comience a restarse de las reuniones).

A continuación se definen las propiedades de la clase ```Persona```, las cuales manejan los cambios de estado de cada persona, es decir, son funciones que van cambiando automáticamente las condiciones de cada atributo de ```Persona```. Modifican la situación de cada instancia persona dadas las condiciones, como por ejemplo desde _S_, que significa que esta sano a _A_ , que sería que este con el virus y esté asintomático, es decir que sea un vector, que luego pase a _E_ que nos dice que ahora sí presenta sintomas, o sea que está enfermo, y que pasen los días y pase a tener como estado _R_ o _M_ que significaría que esta recuperado o muerto respectivamente. Como grupo, agregamos el estado muerto para darle realidad al problema, ya que se está obviando una posibilidad muy real y no debería pasarse por alto.

En esto se evidencia la ventaja de usar Programación Orientada a Objetos (Clases), en que son moldeables, es muy fácil de cambiar estados, clasificarlos y de agregar todo tipo de información de manera ordenada, a diferencia de otras estructuras de datos que son inmutables o más complicadas de acceder a datos, como las tuplas y las listas respectivamente. Además, el uso de memoria de la clase es pequeño, ya que al ser 5 atributos por persona, su memoria es O(5 × _n_), siendo _n_ el número de personas.

Luego creamos dos metodos, **contacto** y **pasar_dia**. El primero, se usa para cuando una persona sana se enfrenta a un posible contagio, como tener contacto con otra persona que es vector de contagio, y consiste en una especie de tirar el dado para crear una variable aleatoria , dejando la posibilidad de contagio como un evento azaroso. En el caso de que sí se contagie, se actualiza el estado de esa persona y las cuentas regresivas se ajustan según sus propios días de recuperacion y días hasta que tenga síntomas. El segundo método de pasar día, solamente se ejecuta en personas que estén enfermas o asintomáticas y no con las sanas o muertas con el fin de no correr la función de manera innecesaria. Y lo que hace es simplemente pasar un día, es decir actualizar las cuentas regresivas que tenga, como si hubiera pasado un día.


In [ ]:
class Persona:
    def __init__(self,id,probabilidad_contagio):
        self._id = id 
        self._probabilidad_contagio = probabilidad_contagio #probabilidad de contagio inicial de cada persona
        self.estado = "S" # S: sano; A asintomatico; E enfermo ; M muerto ; R recuperado
        self._dias_recuperacion = 0 #cuenta regresiva a recuperacion, se inicializan en 0 porque al principio no saben si tienen la enfermedad
        self._dias_sintomas = 0 #cuenta regresiva a presentar sintomas
    @property
    def dias_recuperacion(self):  # manejan los cambios de estado (todas las propertys @)
        return self._dias_recuperacion
    @dias_recuperacion.setter
    def dias_recuperacion(self, a):
        if(a <= 0):
            self._dias_recuperacion = 0
            self.estado = "R"
        self._dias_recuperacion = a
    @property
    def dias_sintomas(self):
        return self._dias_sintomas
    @dias_sintomas.setter
    def dias_sintomas(self, a):    #dias hasta que empiezan a tener sintomas
        if(a <= 0):
            self._dias_sintomas = 0
            self.estado = "E"                    #usamos clases por facilidad de modificar y clasificar estados, agregar todo tipo de informacion
        self._dias_sintomas = a
    def contacto(self,dado,dias_recuperacion,dias_sintomas):        #dado es para ver si se contagio o no, dado el numero del dado
        if self.estado == "S" and dado <= self._probabilidad_contagio:
            self.estado = "A"
            self._dias_recuperacion = dias_recuperacion
            self._dias_sintomas = min(dias_sintomas, dias_recuperacion)

    def pasar_dia (self):
        if self.estado in ["A","E"]:                 #solo afecta si estas asintomatico o enfermo, sino estas sano o muerto, no afecta
            if self.estado == "A":
                self.dias_sintomas -= 1
            self.dias_recuperacion -= 1

    def __repr__(self):                     #los dos para que se vea bonito
        return self.estado+str(self._id)
    def __str__(self):
        if self.estado == "S":
            return f"Soy {self._id}, estoy Sano, P contagio {self._probabilidad_contagio}"
        if self.estado == "A":
            return f"Soy {self._id}, estoy Asintomatico,sintomas:{self._dias_sintomas}, recuperacion {self._dias_recuperacion}"
        if self.estado == "E":
            return f"Soy {self._id}, estoy Enfermo, recuperacion {self._dias_recuperacion}"
        if self.estado == "R":
            return f"Soy {self._id}, estoy Recuperado "

Se nos pide crear la funcion **graficar_infectados** a la cual le entregamos un diccionario con los números de contagiados en cada día y los números de recuperados e infectados hasta la fecha. Nos retorna un gráfico con las curvas que generaron estos datos por día y también guarda este gráfico como un archivo *.png*.

In [ ]:
#Codigo_Graficos.py
import matplotlib.pyplot as plt
import numpy as np


def graficar_infectados(diccionario):
    cantidad_casos_totales = diccionario["it"]   #it es infectados totales
    cantidad_casos_recuperados = diccionario["r"] #recueprados
    cantidad_casos_actuales = diccionario["i"] #infectados actuales
    x = diccionario["t"]   # dia
    plt.plot(x, cantidad_casos_actuales,label= 'casos actuales')
    plt.plot(x, cantidad_casos_recuperados, label= 'casos recuperados')
    plt.plot(x, cantidad_casos_totales, label= 'casos totales')
    plt.ylabel('Cantidad de infectados')
    plt.xlabel('Días')
    plt.title("Infectados en función del tiempo")

    plt.legend(loc=0)
    plt.savefig("grafico_infectados.png")
    mostrar=plt.show()
    return mostrar

# Main script 
## Librerias Y VERBOSIDAD

En cuanto a las librerías, las comentadas no son necesarias en jupyter notebook porque ya se ejecutaron las celdas.
Si verboso es ```True``` se imprimirán cosas en la consola, sino no se irán imprimiendo las simulaciones.

In [ ]:
from collections import defaultdict          #para ordenarse en los momentos de reunion
#from personas import Persona                #clase creada, mencionada en la celda anterior
#from simulation_parameters import simulation_parameters    #para llamar a la funcion simulation_parameters
#from Codigo_Graficos import graficar_infectados            #para llamar a la funcion
import os  #https://stackoverflow.com/questions/7165749/open-file-in-a-relative-location-in-python
import re  #https://stackoverflow.com/questions/1249388/removing-all-non-numeric-characters-from-string-in-python
import copy  #https://www.geeksforgeeks.org/copy-python-deep-copy-shallow-copy/
import time  #medir los tiempos tiempos de simulacion, para medir la eficienci
VERBOSIDAD = 1 # 0 es no imprimir nada, 1 minimo (recomendado), 2 es imprimir lo basico  , 3 es imprimir los detalles

Para poder modelar la evolución temporal del contagio en la red, tuvimos que crear un grafo organizándolo de la siguiente manera: 
* El conjunto de nodos iba a corresponder a cada persona cada día, siendo un nodo la persona 1 el día 1, otro nodo esa misma persona el día 2, puesto que sus parámetros cambiaban y sus relaciones dependían del espacio temporal en el cual se ubicaba (las reuniones se hacían un día especifico). 
* El conjunto de los arcos iba a corresponder a las reuniones entre las personas en cada día: Cada persona se unía con si misma al día siguiente, de manera dirigida, porque el arco dirigido en las reuniones tiene un significado de sentido en la enfermedad, es decir, si la persona 1 tiene el virus y la 2 no, es 1 quien contagia a 2, y no al revés. Además, en el eje del tiempo es más intuitivo el por qué de la dirección del arco, pero este fin radica en ordenar la evolución temporal de la enfermedad y que no se pueda mirar de forma alguna tal que alguien que estaba enfermo el día 2 contagió a una persona el día 1. 


En fin, se puede ver gráficamente esto como un plano cartesiano donde las coordenadas de cada nodo son *(t,n)* donde *t* es el día y *n* el id de la persona. Los arcos unen a las personas que se juntaron ese día. El arco temporal une *(t,n)* con *(t+1,n)* .

Modelar los arcos de esta forma, nos lleva a ahorrar mucho en términos de memoria en el código, ya que almacenamos las reuniones de manera más eficiente. 

Tomemos el ejemplo de que se juntan las personas 1, 2 y 3 el día 1, entonces si se uniera cada persona presente en la reunión entre sí, ocuparíamos 6 arcos en describir esa reunión: *((1,1),(1,2)),((1,1),(1,3)),((1,2),(1,1)),((1,2),(1,3)),((1,3),(1,1)),((1,3),(1,2))*. En cambio, si lo almacenamos en sets, tiene una memoria constante y el set sería solamente ((1,1),(1,2),(1,3)), lo que sería comparar un almacenamiento de orden O(n!) con uno de orden O(n × t), respectivamente.

En el código implementamos, tal como nos pedían, la función **crear_grafo** a la que le damos los archivos de las reuniones y de las personas y nos retorna el conjunto de nodos y de arcos, es decir, nos retorna el grafo.

In [ ]:
#archivo main.py
def crear_grafo ( personas, reuniones):
    """ Crea un default diccionario donde las llaves son los dias y los elementos son una
    lista con los sets con los id de las personas que se juntaron """
    data_reuniones = defaultdict(list)
    with open (reuniones) as file:    
        linea = file.__next__()       
    inter = linea.split("({")         #lee la base de datos

    for i in range(1,len(inter)):
        parcial = set(map(int,inter[i].split("}")[0].split(", ")))    #usamos sets porque es mas eficiente la busqueda, facilita todo
        dia = int(re.sub("[^0-9]", "",inter[i].split("}")[1] ))       
        data_reuniones[dia].append(parcial)
    #reuniones son los arcos
    #cantidad de dia desde la infeccion que su estado de Enfermos con Sintomas
    """ Crea un diccionario donde las llaves son las id y los elementos son instancias de la clase personas"""
    data_personas = dict()    #no es defaultdict porque no tiene sentido preguntar por personas que no existen (gigo)
    with open (personas) as file:   #fue un diccionario en vez de una lista, porque al principio estaba  desordenado y ahora no, pero ya esta hecho
        linea = file.__next__().strip("{}")
    inter = linea.split(", ")
    for i in inter:
        persona = i.split(":")
        id = int(persona[0])
        data_personas[id]= Persona(id,float(persona[1]))   #aca se guardo en cada nodo, una instancia para cada persona
    return (data_personas,data_reuniones)

# aca va el codigo de la funcion
script_dir = os.path.dirname("Informe.ipynb") #<-- absolute dir the script is in
CODIGO = 1500 #Cantidad de pacientes
personas = os.path.join(script_dir, f"Instancias/personas_{CODIGO}.txt")     #aca se crea el grafo (incluyendo las siguientes 3 lineas)
reuniones = os.path.join(script_dir, f"Instancias/reuniones_{CODIGO}.txt")
grafo = crear_grafo(personas,reuniones)  

En esta sección implementamos la función **determinar_contagiados** que recibe nuestro gráfico hecho con la función **crear_grafo**, el *id* del primer infectado, y el número de días transcurridos. Esta función trabaja usando sets que tiene como elementos a las personas posiblemente contagiadas. Usa métodos de los sets como **.add** , **.union**,los cuales tienen una complejidad de O(Log n), y **.intersection** que tiene un orden de complejidad de O(n), además de buscar dentro del mismo set, que tiene O(1); por lo que la función es de orden lineal, lo que la hace bastante eficiente.

El caso que causaría más ineficiencia en la función (y en cualquier otra función del código) es el caso en el que todas las personas se juntaran todos los días sin descanso, ya que elevaría muchísimo la cantidad de memoria y el tiempo de ejecución del programa aletarandolo definitivamente.

In [ ]:
#archivo main.py

#bastante efciciente creo que el peor caso seria si todas las personas se juntaran todos los dias
def determinar_contagiados(grafo, p0 ,delta_dias):
    #grafo es un grafo formado por la estructura de la funcion crear_grafo
    #p0 es in integer con el indice del paciente 0
    #delta_dias número de días transcurridos
    posibles = set()
    posibles.add(p0)
    data_reuniones = grafo[1]
    for t in range (0,delta_dias+1):
        if VERBOSIDAD>1:
            print(f"POSIBLES al principio del DIA {t}",posibles)
        for reunion in data_reuniones[t]:
            if posibles.intersection(reunion):
                posibles = posibles.union(reunion)
                if VERBOSIDAD > 3:
                    print(F"Reunion infectada DIA{t}:",reunion, "culpables:",posibles.intersection(reunion))
    if VERBOSIDAD > 1:
        print(F"\n POSIBLES al final del dia {t}",posibles)
    return posibles

#acá se debería correr la función
script_dir = os.path.dirname("Informe.ipynb") #<-- absolute dir the script is in
CODIGO = 1500 #Cantidad de pacientes
personas = os.path.join(script_dir, f"Instancias/personas_{CODIGO}.txt")     #aca se crea el grafo (incluyendo las siguientes 3 lineas)
reuniones = os.path.join(script_dir, f"Instancias/reuniones_{CODIGO}.txt")
grafo = crear_grafo(personas,reuniones)
p0 = 33 #paciente original
delta_dias = 7 # Cantidad de dias
determinar_contagiados(grafo,p0,delta_dias)

Ahora se nos pide simular los contagios desde el paciente cero (primer enfermo) de dos instancias del problema, pero para ello debemos crear la función **simular_contagio**, la cual recibe el grafo hecho en la parte anterior, el id del paciente cero, el número de días que transcurren en la simulación y la semilla de simulación. En esta usamos la función dada al inicio **simulation_parameters** para tener los parámetros de cada ```Persona```, y utilizando nuevamente sets para los asintomáticos o vectores, para los infectados y para los recuperados, en los cuales vamos agregando a los que entran en ese estado cada día y sacando los que se cambian de estado. La razón por la que utilizamos sets es que tienen una complejidad sumamente baja el revisarlos. De sus métodos el más ineficiente que se usa es **.intersection** que tiene una complejidad de O(len(set)), como hemos mencionado anteriormente.

Es en esta función donde está la verdadera fuente de ineficiencia del código, porque tiene que revisar a todas las personas, con sus atributos, todos los días, lo cual tiene una complejidad de O(n × t). Se podría llevar la cuenta de cambios de estado de cada persona con sumas y restas, y así tener también los estados de cada persona a cada momento, pero así perderíamos la moldeabilidad que nos da la POO, la posibilidad de escalar y proyectar el problema y de poder añadir características y más realidad al problema, tales como relaciones más complejas. Por ejemplo, que dos personas vivan en la misma casa, que hayan más enfermedades, que exista la probabilidad de muerte del infectado, entre muchas otras opciones de escalabilidad en una situación real. Todo eso se puede gracias a estamos tratando un objeto, en vez de una sola variable que podria ser un estado asociado a un número, como por ejemplo *S = 0*. Sin embargo, esta función sigue siendo la fuente de ineficiencia del código, por lo que en el caso de que se quisiera mejorar su eficiencia, es esta la sección que se debe trabajar.

En la simulación, el programa nos va entregando la cantidad de infectados ("Inf:"), la cantidad de vectores ("-Vect:") y el número de personas recuperadas ("-Rec:") por cada día de simulación y los datos de cada día se van guardando en listas, para que queden ordenados cronológicamente, luego se itera en cada reunión y se ejecuta la probabilidad de contagio (como mencionamos antes, similar a tirar un dado, con respecto al número de probabilidad asignado a esa persona). Luego, según el estado de cada persona se va agregando a un set para ir mostrando a medida que avanza la simulación y se crean diccionarios a partir de las listas creadas, para que sirvan de parámetros a la función **graficar_infectados**.

Luego, se procede a realizar las dos simulaciones que se piden en el enunciado, una para las instancias dadas en *personas_15000.txt* y *reuniones_15000.txt* y otra para las instancias dadas en *personas_1500.txt* y *reuniones_1500.txt* donde se simula la situación con la función recién descrita y luego se grafican las curvas cumulativas de contagios, de recuperados y de infectados a cada día, con la función también descrita anteriormente, **graficar_infectados**.


In [ ]:
def simular_contagio( grafo, p0, delta_dias, s):
    #vectores = infectados actuales 
    data_personas = grafo[0]           # primer elemento del grafo (los nodos)
    data_reuniones = grafo[1]          # segundo
    "Se simulan los parametros necesarios para cada persona"
    parametros = simulation_parameters(len(data_personas),delta_dias,s)
    dias_recuperacion = parametros[0]
    dias_sintomas = parametros[1]
    contagios = parametros[2]
    vectores = set()        #los que pueden contagiar
    vectores.add(p0)
    infectados = set()
    infectados.add(p0)
    recuperados = set()

    data_personas[p0].contacto(0,dias_recuperacion[p0] , dias_sintomas[p0])  ##inicilizar el tiempo para p0 con sus contadores
    datos_infectados = []
    datos_recuperados = []
    datos_it = []
    dias = []

    for t in range (0,delta_dias+1):
        if VERBOSIDAD > 1:
            print(f"Comienza DIA {t}")
            print("Inf:",len(infectados),"-Vect:",len(vectores),"-Rec:",len(recuperados))
        datos_infectados.append(len(infectados))
        datos_recuperados.append(len(recuperados))
        datos_it.append(len(recuperados)+len(infectados))     #guardando los datos de cada dia para ponerlos en el grafico
        dias.append(t)
        for reunion in data_reuniones[t]:
            culpables = vectores.intersection(reunion) #son los que infectaron la reunion, que tiene complejidad O(len(reunion))
            if culpables:                             #si es que existen culpables, que es lo mismo que decir len(culpables)>0            
                for posible_contagiado in reunion - culpables:       # sacamos los elementos iguales
                    id = posible_contagiado
                    if id not in recuperados and id not in infectados:   # si no esta en ninguno de los dos, esta sano
                        data_personas[id].contacto(contagios[t][id],dias_recuperacion[id] , dias_sintomas[id])  # ese contagios da el numero del dado
       
        ##########################################################
        ##### VERDADERA FUENTE DE INEFICIENCIA DEL PROGRAMA ######
        ##########################################################
        # porque iteramos sobre todas las personas todos los dias O(n*t)   n:cantidad personas, t: cantidad de días
        # si queremos mejorar la eficiencia del codigo, esto es lo que hay que mejorar
        infectados = set()
        vectores = set()
        recuperados = set()
        for i in range(0,len(data_personas)):                   
            data_personas[i].pasar_dia()
            if data_personas[i].estado == "A":
                vectores.add(i)
                infectados.add(i)
            elif data_personas[i].estado == "E":
                infectados.add(i)
            elif data_personas[i].estado == "R":
                recuperados.add(i)
            elif data_personas[i].estado == "S":
                pass
            else:
                print("ERROR ESTADO ANOMALO")
    if VERBOSIDAD:
        print(f"Fin del DIA {t}")
        print("Inf:",len(infectados),"-Vect:",len(vectores),"-Rec:",len(recuperados))
    return infectados,recuperados,{"t":dias,
            "i":datos_infectados,
            "it":datos_it,
            "r":datos_recuperados}


VERBOSIDAD = 1
script_dir = os.path.dirname("Informe.ipynb") #<-- absolute dir the script is in
CODIGO = 1500 #Cantidad de pacientes
personas = os.path.join(script_dir, f"Instancias/personas_{CODIGO}.txt")     #aca se crea el grafo (incluyendo las siguientes 3 lineas)
reuniones = os.path.join(script_dir, f"Instancias/reuniones_{CODIGO}.txt")
grafo = crear_grafo(personas,reuniones)
p0 = 33 #paciente original
delta_dias = 75 # Cantidad de dias
SEED = 6
resultados = simular_contagio( grafo, 33, delta_dias, SEED)
graficar_infectados(resultados[2])

In [ ]:
VERBOSIDAD = 1
script_dir = os.path.dirname("Informe.ipynb") #<-- absolute dir the script is in
CODIGO = 15000 #Cantidad de pacientes
personas = os.path.join(script_dir, f"Instancias/personas_{CODIGO}.txt")     #aca se crea el grafo (incluyendo las siguientes 3 lineas)
reuniones = os.path.join(script_dir, f"Instancias/reuniones_{CODIGO}.txt")
grafo = crear_grafo(personas,reuniones)
p0 = 33 #paciente original
delta_dias = 68 # Cantidad de dias
SEED = 1113 
resultados = simular_contagio( grafo, 33, delta_dias, SEED)
graficar_infectados(resultados[2])

*Dato: pusimos esa semilla, los dias y el paciente cero a nuestro gusto, se pueden cambiar y probar*

*Para el análisis usamos p0=33 y seed=1113*

Tomando el caso de las instancias 15000 (*personas_15000.txt* y *reuniones_15000.txt*), podemos ver que hay un crecimiento exponencial durante los primeros 15 días (se puede ir haciendo zoom cambiando la cantidad de días en la celda de arriba), y a partir del día 13 aproximadamente, el crecimiento explosivo de infectados actuales empieza a caer, sigue creciendo, pero no de la misma forma, hasta el día 18 aproximadamente que se da el punto crítico, que es el día en el cual hay más casos(11.1140 enfermos y 3706 vectores). Después de ese día la curva de infectados comienza a decrecer de manera abrupta hasta el día 32 aproximadamente, en el que la curva acumulada de infectados tiene pendiente muy pequeña y la curva de recuperados empieza a aplanarse, quedando 2453 infectados y 294 vectores a esa fecha. Finalmente se da por pasada la pandemia al día 68 en el cual ya no quedan ni infectados ni vectores, ya toda la gente que se enfermó debido al virus esta recuperada.

De lo anterior podemos concluir que no se tomaron las medidas correctas para frenar el contagio o "aplanar la curva", ya que tuvo un pico muy alto y se calmó relativamente rápido. En cambio, en la situación real, el crecimiento ha sido mucho más plano. Aún en Chile se está en la etapa de aumento de casos diarios, por lo que no se ha llegado al punto crítico. Si este caso simulado, se diera en la vida real, los hospitales habrían colapsado hace muchos días y esto sería mucho más caótico de lo que está siendo ahora.



Por último, en esta parte de la tarea, estimamos la probabilidad de que cada persona se contagie luego de una cierta cantidad de días usando la instancia 1000 (*personas_1500.txt y reuniones_1500.txt*), y calculamos la probabilidad usando la frecuencia de que cada persona se infecta durante esos días en las 1000 distintas simulaciones hechas con semillas distintas. Recordemos que la probabilidad frecuentista es aquella que se obtiene por el cuociente entre el número de casos favorables (veces que se infectó cada persona) y el número de casos totales (número de simulaciones).

Para poder hacer este cálculo implementamos la **función probabilidad_contagio** que va haciendo una simulación con el grafo, anota quienes se infectaron sumándole 1 a su "contador", que tiene forma de diccionario en el cual el *id* de la persona es la llave y los elementos, inciados en 0, son la cantidad de veces que se ha infectado. Luego cambia de semilla, sobreescribe el grafo y vuelve a iterar, todo esto 1000 veces (en el caso de la tarea, sin embargo, puede variar a gusto de quien lo corra).

Esta función tiene una complejidad menor, ya que su complejidad radica en que son 1000 iteraciones y cada iteración depende del número de nodos que tiene el grafo, es decir, el producto entre el número de dias y de personas que se simula. Por esto, tiene una complejidad aproximada de O(s × n × t ), donde s es la cantidad de simulaciones, n es la cantidad de personas y t es la cantidad de días, y esto sigue siendo lineal, ya que, el conjunto de nodos es un nodo por cada persona por dia, *n × t* (según la definición ya descrita en la tarea), y puesto que *s* es una constante, el orden es de constante × cantidad de nodos, que representa un orden lineal, y por ende, eficiente. 



# ADVERTENCIA: Correr esta celda puede demorar un tiempo (5 min aporx) 

In [ ]:
def probabilidad_contagio(grafo,p0,delta_dias):
    Cantidad = len(grafo[0])       #se obtiene la cantidad de personas
    SIMULACIONES = 1000
    conteos = {i: 0 for i in range(Cantidad)}      #esto crea un diccionario con las llaves siendo el id de la persona, y con todos los elementos en 0
    tiempo_simulaciones = 0                        #crea el tiempo para las simulaciones
    for s in range (0,SIMULACIONES):               #s es la semilla para poder hacer la cantidad de simulaciones requerida (1000), para tener diferentes semillas
        copia_grafo = copy.deepcopy(grafo)         # en vez de crear grafos para cada simulacion, los copiamos y los vamos sobreescribiendo
        if VERBOSIDAD:
            print("-"*45,f"SIMUACION SEMILLA: {s}","-"*45)
        t_inicio=time.time()
        infectados = simular_contagio(copia_grafo, p0, delta_dias,s)[0]   ##aca da la complejidad del algoritmo que es O(|s|*|Cantidad|)  (cada simulacion se demora la cantidad de dias asi que son lo mismo)
        tiempo_simulacion = time.time()-t_inicio
        tiempo_simulaciones += tiempo_simulacion        
        if VERBOSIDAD:             # lo mismo que decir if verbosidad >=1
            print(F"Tiempo de simulacion {tiempo_simulacion}")
        for i in range(Cantidad):
            if i in infectados:
                conteos[i] += 1       #en este for, se va actualizando el valor del diccionario, que nos dice cuantas veces se enfermo cada persona
    for i in range (0, Cantidad):
        conteos[i]= float(conteos[i])/float(SIMULACIONES) #ya esta hecha la division
        if conteos[i] and VERBOSIDAD > 1:
            print(f"Pasinte{i}: probabilidad: {conteos[i]}") #por simplicidad solo printea los que son distintos de 0 
    print(f"Con {Cantidad} personas y {delta_dias} dias se demora {tiempo_simulaciones}s en hacer {SIMULACIONES} simulaciones\nPromedio de simulacion {tiempo_simulaciones/SIMULACIONES} s")
    return conteos

#acá se debería correr la función
script_dir = os.path.dirname("Informe.ipynb") #<-- absolute dir the script is in
CODIGO = 1500 #Cantidad de pacientes
personas = os.path.join(script_dir, f"Instancias/personas_{CODIGO}.txt")     #aca se crea el grafo (incluyendo las siguientes 3 lineas)
reuniones = os.path.join(script_dir, f"Instancias/reuniones_{CODIGO}.txt")
grafo = crear_grafo(personas,reuniones)
p0 = 33 #paciente original
delta_dias = 22 # Cantidad de dias
probabilidades = probabilidad_contagio(grafo,p0,delta_dias)

In [ ]:
probabilidades

In [ ]:
suma_prob=0
for i in probabilidades:
    suma_prob+=probabilidades[i]
prom_prob=suma_prob/1000
print(prom_prob)

En un primer experimento de simulaciones, usamos 75 días para la simulación, pero como el periodo de tiempo fue tan largo, casi todos los pacientes ya se habían recuperado al finalizar, por lo que las probabilidades calculadas tendían a ser 0 o eran muy bajas en todos las personas. 

Luego, probamos las simulaciones con un objetivo de 22 días y obtuvimos probabilidades de infección muy distintas a las anteriores, mucho más altas. Esto sucede debido a que en esa fecha se esperaba que se estuviera cerca del pico de casos de infectados. Es más, en promedio la probabilidad de que estuviera infectada cada persona para ese entonces era de 0.7952 (truncado a 4 decimales), lo que si uno extrapola mediante una mirada frecuentista, significa que practicamente el 80% de la población estaría contagiada.